In [1]:
import sys
sys.path.insert(19, 'litepose-pose-estimation/src')
sys.path.insert(20, 'src/referee_gloves_detector')
import json
import torch
from torchvision import transforms
import lp_config.lp_common_config as cc
from lp_model.lp_litepose import LitePose
from lp_inference.lp_inference import inference, assocEmbedding
from lp_utils.lp_image_processing import drawHeatmap, drawKeypoints, drawSkeleton
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import pandas as pd
from preprocessing import red_filtering, segmentation_and_cropping, equalizing, squaring
import math
from Homography import Homography
import copy
from imutils.video import VideoStream
import time

In [ ]:
# Leggo il file di configurazione
with open('config/config.json') as f:
    config_data = json.load(f)

# Ottengo i percorsi dei file
file_path_big_arch = config_data['path_big_arch']
file_path_csv_keypoints = config_data['path_csv_keypoints']

In [ ]:
model = LitePose().to(cc.config["device"])
model.load_state_dict(torch.load(file_path_big_arch, map_location=cc.config["device"]))

In [ ]:
# HOMOGRAPHY IMAGE AUTOMATICALLY

#uri = "C:\\Users\\stolf\\dev\\Progetto Visione e Percezione\\src\\referee_gloves_detector\\resources\\bloisi_nao_red_hands.png"
#uri = "C:\\Users\\stolf\\Downloads\\test\\test_1.jpg"
#uri = "C:\\Users\\stolf\\Downloads\\test\\test_2.jpg"
uri = "C:\\Users\\stolf\\Downloads\\test\\test_3.jpg"
#uri = "C:\\Users\\stolf\\Downloads\\test\\test_4.jpg"

img = Image.open(uri).convert('RGB')
resize = transforms.Resize([224, 224])
img = resize(img)
to_tensor = transforms.ToTensor()
tensor = to_tensor(img)
tensor = tensor.unsqueeze(0)
output, keypoints = inference(model, tensor)
restricted_keypoints = [[keypoints[0][0], keypoints[0][1], keypoints[0][6], keypoints[0][7], keypoints[0][12],keypoints[0][13]]]
idx = 0
img = drawKeypoints(tensor[idx], restricted_keypoints[idx])
cv2.imshow("Pose estimation", img)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
im_pil = Image.fromarray(img, 'RGB')
im_pil.save('src/referee_gloves_detector/resources/src.jpg')
src = cv2.imread("src/referee_gloves_detector/resources/src.jpg")
dst = cv2.imread("src/referee_gloves_detector/resources/dst.jpg")
h = Homography(src,dst)
#punti2d_old = [[263, 121], [74, 121], [212, 330], [116, 330], [229, 431], [101, 431], [228, 529], [100, 529], [166, 41], [166, 114]]
punti2d = [[412, 271], [226, 271], [363, 481], [264, 481], [317, 191], [317, 266]]
punti3d = []
index_list = [0, 1, 6, 7, 12, 13]
count = 0
index_to_remove = []
for i in index_list:
    try:
        punti3d.append([keypoints[0][i][0]['x'], keypoints[0][i][0]['y']])
    except:
        index_to_remove.append(count)
    finally:
        count = count + 1
for index in sorted(index_to_remove, reverse=True):
    del punti2d[index]
corr = h.normalize_points(punti2d,punti3d)
h._compute_view_based_homography(corr)
plan_view = cv2.warpPerspective(src, h.H, (dst.shape[1], dst.shape[0]))
cv2.imshow("Pose estimation homography", plan_view)
cv2.waitKey()
cv2.destroyAllWindows()

In [7]:
# warm up the camera sensor
cap = cv2.VideoCapture(0)
time.sleep(2.0)
# pip install opencv-contrib-python
fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()

# number of frames is a variable for development purposes, you can change the for loop to a while(cap.isOpened()) instead to go through the whole video
num_frames = 350

first_iteration_indicator = 1
for i in range(0, num_frames):
    '''
    There are some important reasons this if statement exists:
        -in the first run there is no previous frame, so this accounts for that
        -the first frame is saved to be used for the overlay after the accumulation has occurred
        -the height and width of the video are used to create an empty image for accumulation (accum_image)
    '''
    if (first_iteration_indicator == 1):
        ret,frame=cap.read()
        first_frame = copy.deepcopy(frame)
        #print(type(frame))
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        height, width = gray.shape[:2]
        accum_image = np.zeros((height, width), np.uint8)
        first_iteration_indicator = 0
    else:
        ret,frame=cap.read()  # read a frame
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # convert to grayscale

        fgmask = fgbg.apply(gray)  # remove the background

        # for testing purposes, show the result of the background subtraction
        # cv2.imshow('diff-bkgnd-frame', fgmask)

        # apply a binary threshold only keeping pixels above thresh and setting the result to maxValue.  If you want
        # motion to be picked up more, increase the value of maxValue.  To pick up the least amount of motion over time, set maxValue = 1
        thresh = 2
        maxValue = 2
        ret, th1 = cv2.threshold(fgmask, thresh, maxValue, cv2.THRESH_BINARY)
        # for testing purposes, show the threshold image
        # cv2.imwrite('diff-th1.jpg', th1)

        # add to the accumulated image
        accum_image = cv2.add(accum_image, th1)
        # for testing purposes, show the accumulated image
        # cv2.imwrite('diff-accum.jpg', accum_image)

        # for testing purposes, control frame by frame
        # raw_input("press any key to continue")

    # for testing purposes, show the current frame
    # cv2.imshow('frame', gray)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# apply a color map
# COLORMAP_PINK also works well, COLORMAP_BONE is acceptable if the background is dark
color_image = im_color = cv2.applyColorMap(accum_image, cv2.COLORMAP_HOT)
# for testing purposes, show the colorMap image
# cv2.imwrite('diff-color.jpg', color_image)

# overlay the color mapped image to the first frame
result_overlay = cv2.addWeighted(first_frame, 0.7, color_image, 0.7, 0)

# save the final overlay image
cv2.imwrite('diff-overlay.jpg', result_overlay)

# cleanup
cap.release()
cv2.destroyAllWindows()

In [5]:
cap.release()

In [ ]:
# Apro la webcam e elaboro al volo i frame

webcam=cv2.VideoCapture(0)

if not webcam.isOpened():
    raise Exception("Errore nell'apertura della webcam")

resize = transforms.Resize([224, 224])
to_tensor = transforms.ToTensor()
ret,frame=webcam.read()
while ret:

    frame = cv2.flip(frame, 1)
    #-----------------------------PREPROCESSING START HERE-----------------------------
    #RED FILTERING
    full_mask = red_filtering(frame)
    #SEGMENTATION E CROPPING
    # TODO: migliorare il meccanismo di cropping se deve tagliare più di 224 meglio skippare il frame(creerebbe frame rettangolari e andrebbero in input alla rete immagini distorte)
    cropped_image = segmentation_and_cropping(frame, full_mask)
    #NORMALIZATION
    normalized_image = cv2.normalize(cropped_image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    #EQUALIZATION
    equalized_image = equalizing(normalized_image)
    #------------------------------PREPROCESSING END HERE------------------------------
    img = cv2.cvtColor(equalized_image, cv2.COLOR_BGR2RGB)
    img = squaring(img)

    try:
        # Può dare problemi se gli arriva una immagine con una dimensione mancante,
        # non ho capito il perchè (probabilmente è causata dall'algoritmo di cropping)
        # ma questo può succedere
        im_pil = Image.fromarray(img)
    except:
        print("Shape dell'immagine che farebbe crashare il processo: " + str(img.shape))
        pass #TODO: controllare come va con il continue(per skippare un ciclo in caso di cropping fatto male)
    frame = resize(im_pil)
    tensor = to_tensor(frame)
    tensor = tensor.unsqueeze(0)

    if ret==True:
        output, keypoints = inference(model, tensor)
        restricted_keypoints = [[keypoints[0][0], keypoints[0][1], keypoints[0][6], keypoints[0][7], keypoints[0][12],keypoints[0][13]]]
        embedding = assocEmbedding(keypoints)
        #frame = drawSkeleton(tensor[0], embedding[0])
        frame = drawKeypoints(tensor[0], restricted_keypoints[0])
        cv2.imshow("Pose estimation", frame)

        #-----------------------------HOMOGRAPHY START HERE-----------------------------
        # TODO: inserire treshold sul reprojection error(stavo pensando di impostarla a 0.1)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image_pil = Image.fromarray(frame, 'RGB')
        image_pil.save('src/referee_gloves_detector/resources/src.jpg')
        src = frame
        dst = cv2.imread("src/referee_gloves_detector/resources/dst.jpg")
        h = Homography(src,dst)
        punti2d = [[412, 271], [226, 271], [363, 481], [264, 481], [317, 191], [317, 266]]
        punti3d = []
        index_list = [0, 1, 6, 7, 12, 13]
        count = 0
        index_to_remove = []
        for i in index_list:
            try:
                punti3d.append([keypoints[0][i][0]['x'], keypoints[0][i][0]['y']])
            except:
                index_to_remove.append(count)
            finally:
                count = count + 1
        for index in sorted(index_to_remove, reverse=True):
            del punti2d[index]
        corr = h.normalize_points(punti2d,punti3d)
        h._compute_view_based_homography(corr)
        plan_view = cv2.warpPerspective(src, h.H, (dst.shape[1], dst.shape[0]))
        plan_view = cv2.cvtColor(plan_view, cv2.COLOR_BGR2RGB)
        cv2.imshow("Pose estimation homography", plan_view)
        #------------------------------HOMOGRAPHY END HERE------------------------------

        key=cv2.waitKey(1) & 0xFF
        if key==ord("q"):
            break
    ret, frame = webcam.read()
webcam.release()
cv2.destroyAllWindows()